In [28]:
import pandas as pd
import numpy as np
import os
from binance.client import Client

In [29]:
#### ocultar dados ######
api_key = os.environ.get('API_KEY')
api_secret = os.environ.get('API_SECRET')

client=Client(api_key,api_secret)

In [56]:
def GetOHLC(asset:'str',timeframe,start,end):
    df= pd.DataFrame(client.get_historical_klines(asset, timeframe,start,end))
    df=df.iloc[:,:6]
    df.columns=["Date","Open","High","Low","Close","Volume"]
    df=df.set_index("Date")
    df.index=pd.to_datetime(df.index,unit="ms")
    df=df.astype("float")
    return df

def intersecao_dfs(dic):
  chaves = list(dic.keys())
  init = dic[chaves[0]].index.intersection(dic[chaves[0]].index)
  for k in chaves[1:]:
    init = init.intersection(dic[k].index)
  return init


def get_good_dfs(dic,usar):
  for k in dic.keys():
    df = dic[k]
    df = df[df.index.isin(usar)].copy()
    dic[k] = df
  return dic
  

In [45]:
data_set_4h = {}
data_set_1h = {}
data_set_1d = {}

tickers = ['BTCUSDT','ETHUSDT','BNBUSDT','ADAUSDT','XRPUSDT','LTCUSDT']


for t in tickers:
  df = GetOHLC(t,'4h','2017-01-01','2023.11.01')
  data_set_4h[t] = df


for t in tickers:
  df = GetOHLC(t,'1d','2017-01-01','2023.11.01')
  data_set_1d[t] = df


for t in tickers:
  df = GetOHLC(t,'1h','2017-01-01','2023.11.01')
  data_set_1h[t] = df



In [62]:
usar_4h = intersecao_dfs(data_set_4h)
usar_1h = intersecao_dfs(data_set_1h)
usar_1d = intersecao_dfs(data_set_1d)

In [63]:
data_set_4h_tratado = get_good_dfs(data_set_4h,usar_4h)
data_set_1h_tratado = get_good_dfs(data_set_1h,usar_1h)
data_set_1d_tratado = get_good_dfs(data_set_1d,usar_1d)

In [64]:
for k in data_set_4h_tratado.keys():
  df = data_set_4h_tratado[k]
  df.to_excel(f'./assets/4h/{k}.xlsx')


In [ ]:
for k in data_set_1h_tratado.keys():
  df = data_set_1h_tratado[k]
  df.to_excel(f'./assets/1h/{k}.xlsx')

In [ ]:
for k in data_set_1d_tratado.keys():
  df = data_set_1d_tratado[k]
  df.to_excel(f'./assets/1d/{k}.xlsx')